In [1]:
import torch

In [2]:
import torch.autograd as autograd

In [3]:
import torch.nn as nn

In [4]:
import torch.nn.functional as F

In [5]:
import torch.optim as optim

In [6]:
torch.manual_seed(1)

In [7]:
word_to_ix = {"hello": 0, "world": 1}

In [8]:
embeds = nn.Embedding(2, 5)

In [9]:
print(embeds)

Embedding(2, 5)


In [10]:
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])

In [11]:
hello_embed = embeds(autograd.Variable(lookup_tensor))

In [12]:
print(hello_embed)

Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



In [13]:
CONTEXT_SIZE = 2

In [14]:
EMBEDDING_DIM = 10

In [15]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [16]:
print(test_sentence)

['When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', 'thy', "beauty's", 'field,', 'Thy', "youth's", 'proud', 'livery', 'so', 'gazed', 'on', 'now,', 'Will', 'be', 'a', "totter'd", 'weed', 'of', 'small', 'worth', 'held:', 'Then', 'being', 'asked,', 'where', 'all', 'thy', 'beauty', 'lies,', 'Where', 'all', 'the', 'treasure', 'of', 'thy', 'lusty', 'days;', 'To', 'say,', 'within', 'thine', 'own', 'deep', 'sunken', 'eyes,', 'Were', 'an', 'all-eating', 'shame,', 'and', 'thriftless', 'praise.', 'How', 'much', 'more', 'praise', "deserv'd", 'thy', "beauty's", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'of', 'mine', 'Shall', 'sum', 'my', 'count,', 'and', 'make', 'my', 'old', "excuse,'", 'Proving', 'his', 'beauty', 'by', 'succession', 'thine!', 'This', 'were', 'to', 'be', 'new', 'made', 'when', 'thou', 'art', 'old,', 'And', 'see', 'thy', 'blood', 'warm', 'when', 'thou', "feel'st", 'it', 'cold.']


In [17]:
print(len(test_sentence))

115


In [18]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2]) for i in range(len(test_sentence) - 2)]

In [19]:
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [20]:
vocab = set(test_sentence)
word_to_ix ={word: i for i, word in enumerate(vocab)}

In [21]:
print(vocab)

{'Were', 'sum', "'This", 'lies,', 'old', "excuse,'", 'blood', 'lusty', 'asked,', 'livery', 'old,', 'on', 'brow,', 'This', 'my', 'proud', 'Thy', 'mine', 'his', 'eyes,', 'days;', 'count,', 'be', 'thy', 'beauty', 'a', 'dig', 'To', "deserv'd", 'thine', 'And', 'Where', 'child', "beauty's", 'thriftless', 'to', 'within', 'and', 'were', 'treasure', 'made', 'cold.', 'shall', 'praise', 'in', 'How', 'use,', 'see', 'the', 'Shall', 'by', 'field,', 'Proving', 'say,', 'when', 'being', 'Then', 'more', 'Will', 'sunken', 'forty', 'all-eating', 'much', 'it', 'warm', "youth's", 'gazed', 'When', 'deep', 'thine!', "feel'st", 'so', 'worth', 'If', 'small', 'trenches', 'an', 'answer', 'shame,', 'all', 'couldst', 'make', 'art', 'new', 'thou', 'succession', 'where', 'own', 'winters', 'besiege', 'praise.', 'fair', "totter'd", 'of', 'now,', 'held:', 'weed'}


In [22]:
print(word_to_ix)
print(len(word_to_ix))

{'praise': 43, 'in': 44, 'This': 13, 'Were': 0, 'winters': 88, 'fair': 91, 'How': 45, 'use,': 46, 'sum': 1, "'This": 2, 'lies,': 3, 'see': 47, 'the': 48, 'Shall': 49, 'blood': 6, 'by': 50, 'thriftless': 34, 'field,': 51, 'Proving': 52, 'thine!': 69, 'say,': 53, 'when': 54, 'asked,': 8, 'being': 55, 'couldst': 80, 'Thy': 16, 'livery': 9, 'Then': 56, 'on': 11, 'brow,': 12, "deserv'd": 28, 'more': 57, 'Will': 58, 'my': 14, 'sunken': 59, 'proud': 15, 'forty': 60, 'all-eating': 61, 'his': 18, 'eyes,': 19, 'mine': 17, 'much': 62, 'it': 63, 'lusty': 7, 'warm': 64, 'shame,': 78, 'praise.': 90, 'beauty': 24, 'all': 79, 'days;': 20, 'gazed': 66, 'count,': 21, 'When': 67, 'deep': 68, 'be': 22, 'thy': 23, "feel'st": 70, 'so': 71, 'worth': 72, 'If': 73, 'small': 74, 'trenches': 75, 'a': 25, 'own': 87, 'an': 76, 'answer': 77, 'were': 38, 'dig': 26, 'To': 27, 'old,': 10, 'art': 82, 'make': 81, 'new': 83, 'thou': 84, 'succession': 85, 'where': 86, 'thine': 29, 'And': 30, 'Where': 31, 'child': 32, "you

In [23]:
for i, word in enumerate(vocab):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


In [24]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs
    

In [25]:
losses = []

In [26]:
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [27]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        model.zero_grad()
        
        log_probs = model(context_var)
        
#         loss = loss_function(log_probs, autograd.Variable(torch.LongTensor[word_to_ix[target]]))
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))    
    
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 520.7652
[torch.FloatTensor of size 1]
, 
 518.3556
[torch.FloatTensor of size 1]
, 
 515.9653
[torch.FloatTensor of size 1]
, 
 513.5928
[torch.FloatTensor of size 1]
, 
 511.2362
[torch.FloatTensor of size 1]
, 
 508.8963
[torch.FloatTensor of size 1]
, 
 506.5715
[torch.FloatTensor of size 1]
, 
 504.2612
[torch.FloatTensor of size 1]
, 
 501.9653
[torch.FloatTensor of size 1]
, 
 499.6832
[torch.FloatTensor of size 1]
]


In [28]:
CONTEXT_SIZE = 2
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [29]:
vocab = set(raw_text)

In [30]:
vocab

{'As',
 'Computational',
 'In',
 'People',
 'The',
 'We',
 'a',
 'about',
 'abstract',
 'are',
 'beings',
 'by',
 'called',
 'computational',
 'computer',
 'computers.',
 'conjure',
 'create',
 'data.',
 'direct',
 'directed',
 'effect,',
 'evolution',
 'evolve,',
 'idea',
 'inhabit',
 'is',
 'manipulate',
 'of',
 'other',
 'our',
 'pattern',
 'process',
 'process.',
 'processes',
 'processes.',
 'program.',
 'programs',
 'rules',
 'spells.',
 'spirits',
 'study',
 'that',
 'the',
 'they',
 'things',
 'to',
 'we',
 'with'}

In [31]:
vocab_size = len(vocab)
print(vocab_size)

49


In [32]:
word_to_ix = {word:i for i, word in enumerate(vocab)}
data = []
print(word_to_ix)
print(len(raw_text))

{'effect,': 0, 'things': 1, 'The': 2, 'beings': 3, 'In': 6, 'inhabit': 5, 'we': 7, 'conjure': 8, 'that': 9, 'processes.': 10, 'are': 11, 'Computational': 12, 'evolution': 13, 'programs': 14, 'the': 15, 'with': 16, 'a': 17, 'directed': 18, 'spirits': 19, 'abstract': 20, 'computational': 22, 'is': 23, 'by': 24, 'People': 25, 'other': 26, 'to': 38, 'idea': 28, 'our': 27, 'As': 29, 'processes': 30, 'pattern': 31, 'computers.': 32, 'create': 33, 'manipulate': 4, 'called': 34, 'We': 35, 'rules': 36, 'evolve,': 37, 'data.': 39, 'process.': 40, 'spells.': 41, 'of': 42, 'process': 43, 'computer': 44, 'direct': 21, 'study': 45, 'they': 46, 'program.': 47, 'about': 48}
62


In [33]:
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [34]:
# class CBOW(nn.Module):

#     def __init__(self, vocab_size, embedding_dim, context_size):
#         super(CBOW, self).__init__()
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.linear1 = nn.Linear(context_size * embedding_dim, 128)
#         self.linear2 = nn.Linear(128, vocab_size)
        
#     def forward(self, inputs):
#         embeds = self.embeddings(inputs).view((1, -1))
#         out = F.relu(self.linear1(embeds))
#         out = self.linear2(out)
#         log_probs = F.log_softmax(out)
#         return log_probs
        

In [35]:
# def make_context_vector(context, word_to_ix):
#     idxs = [word_to_ix[w] for w in context]
#     tensor = torch.LongTensor(idxs)
#     return autograd.Variable(tensor)

# make_context_vector(data[0][0], word_to_ix)

In [36]:
# emirceyani code cbow

In [37]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).sum(dim=0).view((1, -1))
        out = self.linear1(embeds)
        log_probs = F.log_softmax(out)
        return log_probs

In [38]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

context = make_context_vector(data[0][0], word_to_ix)

In [39]:
# create your model and train. here are some functions to help you make
# the data ready for use by your module
losses = []
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [40]:
for epoch in range(200):
    total_loss = torch.Tensor([0])
    for context, target in data:
        
        # step 1: prepare the inputs to be passed to the model
        # into integer indices and wrap them in variables
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        # step 2: 
        model.zero_grad()
        
        # step 3: run the forward pass
        log_probs = model(context_var)
        
        # step 4: compute your loss function
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        
        # step 5: do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 271.3236
[torch.FloatTensor of size 1]
, 
 267.8741
[torch.FloatTensor of size 1]
, 
 264.5412
[torch.FloatTensor of size 1]
, 
 261.3120
[torch.FloatTensor of size 1]
, 
 258.1763
[torch.FloatTensor of size 1]
, 
 255.1258
[torch.FloatTensor of size 1]
, 
 252.1538
[torch.FloatTensor of size 1]
, 
 249.2545
[torch.FloatTensor of size 1]
, 
 246.4231
[torch.FloatTensor of size 1]
, 
 243.6559
[torch.FloatTensor of size 1]
, 
 240.9492
[torch.FloatTensor of size 1]
, 
 238.2999
[torch.FloatTensor of size 1]
, 
 235.7055
[torch.FloatTensor of size 1]
, 
 233.1637
[torch.FloatTensor of size 1]
, 
 230.6723
[torch.FloatTensor of size 1]
, 
 228.2297
[torch.FloatTensor of size 1]
, 
 225.8340
[torch.FloatTensor of size 1]
, 
 223.4838
[torch.FloatTensor of size 1]
, 
 221.1778
[torch.FloatTensor of size 1]
, 
 218.9147
[torch.FloatTensor of size 1]
, 
 216.6932
[torch.FloatTensor of size 1]
, 
 214.5125
[torch.FloatTensor of size 1]
, 
 212.3713
[torch.FloatTensor of size 1]
, 
 210.2688

In [41]:
acc = 0

In [42]:
print(data)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea'), (['the', 'idea', 'a', 'computational'], 'of'), (['idea', 'of', 'computational', 'process.'], 'a'), (['of', 'a', 'process.', 'Computational'], 'computational'), (['a', 'computational', 'Computational', 'processes'], 'process.'), (['computational', 'process.', 'processes', 'are'], 'Computational'), (['process.', 'Computational', 'are', 'abstract'], 'processes'), (['Computational', 'processes', 'abstract', 'beings'], 'are'), (['processes', 'are', 'beings', 'that'], 'abstract'), (['are', 'abstract', 'that', 'inhabit'], 'beings'), (['abstract', 'beings', 'inhabit', 'computers.'], 'that'), (['beings', 'that', 'computers.', 'As'], 'inhabit'), (['that', 'inhabit', 'As', 'they'], 'computers.'), (['inhabit', 'computers.', 'they', 'evolve,'], 'As'), (['computers.', 'As', 'evolve,', 'processes']

In [44]:
for context, target in data:
    # step 1: prepare the inputs to be passed to the model
    context_idxs = [word_to_ix[w] for w in context]
    context_var = autograd.Variable(torch.LongTensor(context_idxs))
    log_probs = model(context_var)
    _, idx = torch.min(log_probs, -1)
    print (context, word_to_ix[target], idx.data[0])

['We', 'are', 'to', 'study'] 48 5
['are', 'about', 'study', 'the'] 38 5
['about', 'to', 'the', 'idea'] 45 36
['to', 'study', 'idea', 'of'] 15 5
['study', 'the', 'of', 'a'] 28 33
['the', 'idea', 'a', 'computational'] 42 33
['idea', 'of', 'computational', 'process.'] 17 39
['of', 'a', 'process.', 'Computational'] 22 38
['a', 'computational', 'Computational', 'processes'] 40 38
['computational', 'process.', 'processes', 'are'] 12 47
['process.', 'Computational', 'are', 'abstract'] 30 23
['Computational', 'processes', 'abstract', 'beings'] 11 23
['processes', 'are', 'beings', 'that'] 20 22
['are', 'abstract', 'that', 'inhabit'] 3 22
['abstract', 'beings', 'inhabit', 'computers.'] 9 23
['beings', 'that', 'computers.', 'As'] 5 48
['that', 'inhabit', 'As', 'they'] 32 26
['inhabit', 'computers.', 'they', 'evolve,'] 29 36
['computers.', 'As', 'evolve,', 'processes'] 46 38
['As', 'they', 'processes', 'manipulate'] 37 36
['they', 'evolve,', 'manipulate', 'other'] 30 36
['evolve,', 'processes', 'o